### We declare a function that reads the files and seperate each image

In [1]:
import numpy as np


def read_images(filename_):
    images = []
    image = []
    
    file_ = open(filename_, "r")
    row_number = 0  #counter to keep track number of rows (Every 28 rows = 1 image)
    
    for x in file_.readlines():
        image.append(x[:28])
        row_number+=1
        if(row_number == 28):
            images.append(image)
            row_number = 0
            image = []
    return images


def read_labels(filename_):
    labels = []
    file_ = open(filename_)
    for x in file_.readlines():
        labels.append(int(x.strip()))
    return labels


### Read images and labels

In [2]:
training_images_ = read_images('trainingimages')
validation_images_ = read_images("validationimages")
test_images_ = read_images('testimages')

training_labels = read_labels('traininglabels')
validation_labels = read_labels("validationlabels")
test_labels = read_labels("testlabels")

# A - Naive Bayes Algorithm
## (i)

### Build probability list for each pixel

In [3]:
from collections import Counter
total_num_images_of_digit = Counter(training_labels) ##Frequency of every digit

def build_dict_pixels():
    dict_pixels = {}
    for i in range(28):
        for j in range(28):
            dict_pixels[str(i)+','+str(j)] = np.zeros(10) #Declare frequency array of size 10 for the digits
    return dict_pixels

pixels_black_prob = build_dict_pixels()

for index, image in enumerate(training_images_): #For every black pixel increase the counter for the right digit(label)
    for i , row in enumerate(image):
        for j , col in enumerate(row):
            if col == '#' or col == '+':
                pixels_black_prob[str(i)+','+str(j)][training_labels[index]] += 1


### Calculate Probabilites for every image

In [4]:
import math
import copy
def cal_naive_bayes(image , k):
    black_pixels = copy.deepcopy(pixels_black_prob)
    
    #Applying Laplace Smoothin with k value
    for digits in black_pixels.values():
        for index , value in enumerate(digits):
            digits[index] = (value + k) / (total_num_images_of_digit[index] + (2*k))
        
    digits_prob = [0] * 10
    for indexI , row in enumerate(image):
        for indexJ , col in enumerate(row):
            for i in range(10):
                if col == '+' or col == '#':
                    digits_prob[i] += math.log10(black_pixels[str(indexI)+','+str(indexJ)][i])
                else:
                    digits_prob[i] += math.log10( 1 - black_pixels[str(indexI)+','+str(indexJ)][i])


    for index , digit in enumerate(digits_prob):
        digits_prob[index] += math.log10(0.1)
        
    max_value = max(digits_prob)
    return digits_prob.index(max_value)



### Function Classify  that apply cal_naive_bayes method on all images and return a list of predicted labels
### We also declare a function get_accuracy to calculate the accuracy of our classifier

In [5]:
# Classify function: receives image dataset and k value for smoothing
# We run through the images, for every image we use naive bayes classifier to predict the label
# Returns the predicted labels
def classify(image_list , k):
    list_to_return = []
    for image in image_list:
        list_to_return.append(cal_naive_bayes(image , k))
    return list_to_return


# Check how many predicted labels were right predicted
def get_accuracy(result_list , label_list):
    counter = 0
    for index , predicted in enumerate(result_list):
        if predicted == label_list[index]:
            counter+=1
    
    return counter / len(result_list)



In [6]:
validate_data_k_1 = classify(validation_images_,1)
validate_data_k_2 = classify(validation_images_,2)
validate_data_k_3 = classify(validation_images_,3)
validate_data_k_4 = classify(validation_images_,4)
validate_data_k_5 = classify(validation_images_,5)

In [7]:
def print_acc(accuracy,k):
    print("The accuracy of the classifier with Smoothing Value K={} is: {}".format(k,accuracy))
    

print_acc(get_accuracy(validate_data_k_1 , validation_labels) , 1)
print_acc(get_accuracy(validate_data_k_2 , validation_labels) , 2)
print_acc(get_accuracy(validate_data_k_3 , validation_labels), 3)
print_acc(get_accuracy(validate_data_k_4 , validation_labels) , 4)
print_acc(get_accuracy(validate_data_k_5 , validation_labels), 5)


The accuracy of the classifier with Smoothing Value K=1 is: 0.818
The accuracy of the classifier with Smoothing Value K=2 is: 0.819
The accuracy of the classifier with Smoothing Value K=3 is: 0.812
The accuracy of the classifier with Smoothing Value K=4 is: 0.811
The accuracy of the classifier with Smoothing Value K=5 is: 0.81


#### For  k=2 we got  the highest accuracy, therefore we choose it to be our smoothing value for next sections.

## (ii)

In [8]:
k = 2
predict_test_data = classify(test_images_ , k )
print("The prediction accuracy for test data is: " , get_accuracy(predict_test_data , test_labels))

The prediction accuracy for test data is:  0.766


## (iii)

In [9]:
predict_training_data = classify(training_images_ , k )
print("The prediction accuracy for training data is: " , get_accuracy(predict_training_data , training_labels))

The prediction accuracy for training data is:  0.8378


-------------------------
# B - Perceptron Algorithm

### Convert all images to 0 and 1. ( 1 when pixel is # or + , 0 when empty)

In [3]:
import numpy as np

def convert2d_to_1d(images):
    images_to_return = []
    for image in images:
        images_to_return.append(image.flatten())
    return images_to_return
        
def image_convertion(images):
    for index , image in enumerate(images):
        for i , row in enumerate(image):
            for j , col in enumerate(row):
                row = list(row)
                if col == '#' or col == '+':
                    row[j] = 1
                else:
                    row[j] = 0
            image[i] = row
        images[index] = np.array(image)
    
    return convert2d_to_1d(images)
    
    

converted_training_images = image_convertion(training_images_)
converted_tested_images = image_convertion(test_images_)
converted_validation_images = image_convertion(validation_images_)

### After converting all images to the right format, we now prepare the training dataset, we do that by:
- For every digit (category) we run through all the training set and set 1 for every image that matches the label , 0 otherwise


In [4]:
learning_rate = 1 #Thats what is requested 
threshold = 0    #Since all weights are Integers that lay between INF and -INF we set threshold to be 0



def prepare_training_dataset(training_set , label):
    data = []
    for index , pixels in enumerate(training_set):
        fire = 0
        if label == training_labels[index]:
            fire = 1
        image_and_label = (tuple(pixels) , fire )
        data.append(image_and_label)
    return data


## We declare weights to be 0 (That what is requested in the question)
#- We run in a loop of epochs range.
#- for every tuple that consist of image (list of pixels) and a expected label:
#  we calculate the dot product of weights vector and image pixels
# if the result is above the threshold then we fire 1 and check if there is an error
# if error exists , update weights with delta rule (OldWeight + Learning rate * (expected_label - result) * pixel_value[1,0])
# Once we have reached max epochs , we return weights
def perceptron_train(images , epochs):
    weights = [0] * len(images[0][0])
    for epoch in range(epochs):
        err_count = 0
        for pixel_vec , exp_label in images:
            res = 0
            if np.dot(pixel_vec , weights) > threshold:
                res = 1
            e = exp_label - res
            if e != 0:
                err_count+=1
                for index , pixel_value in enumerate(pixel_vec):
                    weights[index] += learning_rate * pixel_value * e

    return weights



## (i)

In [5]:
epochs = 3
digit_weights = dict.fromkeys(range(10) , [])

for digit in digit_weights.keys():
    digit_weights[digit] = perceptron_train(prepare_training_dataset(converted_training_images , digit),epochs)
    print("Training for digit:" , digit, 'is done.' )


Training for digit: 0 is done.
Training for digit: 1 is done.
Training for digit: 2 is done.
Training for digit: 3 is done.
Training for digit: 4 is done.
Training for digit: 5 is done.
Training for digit: 6 is done.
Training for digit: 7 is done.
Training for digit: 8 is done.
Training for digit: 9 is done.


### Lets examine how good our training was.

In [6]:
#Classify Function: 
# it receives dataset of images and run through all the digits weights we have already calculated in training,
# for every digit weight, get the score of the dot product and add it to a list of size 10 (digits)
# for every image, return the label with the highest score
def classify(data_images):
    predicted_labels = []
    for image in data_images:
        max_score = []
        for digit, weights in digit_weights.items():
            score = np.dot(weights , image)
            max_score.append(score)
        predicted_labels.append(max_score.index(max(max_score)))
    return predicted_labels


# Check how many predicted labels were right predicted
def get_accuracy(predicted_labels , expected_labels):
    counter = 0
    for exp_label , pred_lab in zip(expected_labels,predicted_labels):
        if exp_label == pred_lab:
            counter+=1
    return counter/len(expected_labels)



train_acc = get_accuracy(classify(converted_training_images) , training_labels)
print("Accuracy for training set with epochs set to 3 has reached: ", train_acc)
test_acc = get_accuracy(classify(converted_tested_images) , test_labels)
print("Accuracy for test set has with epochs set to 3 reached: ", test_acc)



Accuracy for training set with epochs set to 3 has reached:  0.8914
Accuracy for test set has with epochs set to 3 reached:  0.793


## (ii)

In [7]:
def train_with_epochs(epochs):
    for digit in digit_weights.keys():
        digit_weights[digit] = perceptron_train(prepare_training_dataset(converted_training_images , digit),epochs)
        


### Epoch = 1

In [8]:
train_with_epochs(epochs=1)
train_acc = get_accuracy(classify(converted_training_images) , training_labels)
print("Accuracy for training set with epochs set to 1 has reached: ", train_acc)
test_acc = get_accuracy(classify(converted_tested_images) , test_labels)
print("Accuracy for test set has with epochs set to 1 reached: ", test_acc)

Accuracy for training set with epochs set to 1 has reached:  0.8718
Accuracy for test set has with epochs set to 1 reached:  0.804


### Epoch = 2

In [9]:
train_with_epochs(epochs=2)
train_acc = get_accuracy(classify(converted_training_images) , training_labels)
print("Accuracy for training set with epochs set to 2 has reached: ", train_acc)
test_acc = get_accuracy(classify(converted_tested_images) , test_labels)
print("Accuracy for test set has with epochs set to 2 reached: ", test_acc)

Accuracy for training set with epochs set to 2 has reached:  0.8902
Accuracy for test set has with epochs set to 2 reached:  0.803


### Epoch = 4

In [10]:
train_with_epochs(epochs=4)
train_acc = get_accuracy(classify(converted_training_images) , training_labels)
print("Accuracy for training set with epochs set to 4 has reached: ", train_acc)
test_acc = get_accuracy(classify(converted_tested_images) , test_labels)
print("Accuracy for test set has with epochs set to 4 reached: ", test_acc)

Accuracy for training set with epochs set to 4 has reached:  0.9008
Accuracy for test set has with epochs set to 4 reached:  0.79


### Epoch = 5

In [11]:
train_with_epochs(epochs=5)
train_acc = get_accuracy(classify(converted_training_images) , training_labels)
print("Accuracy for training set with epochs set to 5 has reached: ", train_acc)
test_acc = get_accuracy(classify(converted_tested_images) , test_labels)
print("Accuracy for test set has with epochs set to 5 reached: ", test_acc)

Accuracy for training set with epochs set to 5 has reached:  0.906
Accuracy for test set has with epochs set to 5 reached:  0.796
